### Import `requests` library
This package allows you to get any website's HTML code so that you can extract from it. Let's save the website's URL in the `URL` variable.

In [1]:
import requests

URL="https://bulbapedia.bulbagarden.net/wiki/List_of_Pok%C3%A9mon_by_National_Pok%C3%A9dex_number"

### Load the page

In [2]:
page=requests.get(URL)

### Parse HTML data

In [3]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(page.content, 'html.parser')

### Find all tables that contain Pokemon details

In [4]:
# Get main content <div>
poke_content=soup.find(id='mw-content-text')

# Get all <table> elements
poke_tables=poke_content.find_all('table')

### Save them in a JSON

In [7]:
info_start = 3

import json

pokemon_json = []

for j in range(1, 9):
  gen_list = poke_tables[j]
  gen_array = []

  for i in range(info_start, len(gen_list.contents), 2):
      poke_info=gen_list.contents[i]
      kdex=poke_info.contents[1].text.strip()
      ndex=poke_info.contents[3].text.strip()
      name=poke_info.contents[7].text.strip()
      type1=poke_info.contents[9].text.strip()

      if len(poke_info.contents) > 10:
          type2=poke_info.contents[11].text.strip()
          gen_array.append({
              "kdex": kdex,
              "ndex": ndex,
              "name": name,
              "type1": type1,
              "type2": type2
          })
      else:
          gen_array.append({
              "kdex": kdex,
              "ndex": ndex,
              "name": name,
              "type1": type1
          })

  key_name = "generation_" + str(j)
  pokemon_json.append({
      key_name: gen_array
  })

with open('pokemon.json', 'w') as f:
  json.dump(pokemon_json, f)

# Getting news articles

### Import `requests` library
This package allows you to get any website's HTML code so that you can extract from it. Let's save the website's URL in the `URL` variable.

In [8]:
import requests

URL="https://edition.cnn.com/article/sitemap-2021-3.html?fbclid=IwAR2FVSX87L5cGSae9RmGBw34kyVB-YHsTaFxpNdePuGqW5Du1kjD5j_SAi0"

### Load the page

In [9]:
page=requests.get(URL)

### Parse HTML data

In [10]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(page.content, 'html.parser')

### Find all news articles on March 11 and 12, 2021 and save it in a JSON file

In [11]:
import json

body=soup.find_all(class_ = 'sitemap-entry')
news_articles = body[1].find_all('li')

news_json = []

for i in range(len(news_articles)):
  articles = news_articles[i]
  if articles.find(class_ = "date").text.strip() == '2021-03-12' or articles.find(class_ = "date").text.strip() == '2021-03-11':
    article_url = articles.find('a')['href']
    article_page = requests.get(article_url)
    temp_soup = BeautifulSoup(article_page.content, 'html.parser')
    full_article = temp_soup.find(class_ = "pg-rail-tall__wrapper").find(class_ = "pg-rail-tall__body").find('section').find(class_ = "l-container").find_all(class_ = "zn-body__paragraph")
    
    full_article_string = ''

    for article in full_article:
      full_article_string = full_article_string + article.text.strip() + "\n"

    date = articles.find(class_ = "date").text.strip()
    title = temp_soup.find(class_ = 'pg-headline').text.strip()
    author = temp_soup.find(class_ = 'metadata').find('span').text.strip()
    author = author.replace('By ', '')
    author = author.replace(', CNN', '')

    news_json.append({
        "title": title,
        "author": author,
        "date": date,
        "full_article": full_article_string
    })

with open('news.json', 'w') as f:
  json.dump(news_json, f)